<a href="https://colab.research.google.com/github/geovalexis/TFG/blob/main/Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data retrievement


## Reference Dataset

In [21]:
# Quest for Orthologs (QfO) dataset
!ls drive/MyDrive/TFG/QFO_2018/

# Human dataset
!pip install biopython
from Bio import SeqIO
import pandas as pd
records = list(SeqIO.parse("drive/MyDrive/TFG/QFO_2018/UP000005640_9606.xml", "seqxml"))
records_ids = [record.id for record in records]
df_records_ids = pd.DataFrame(records_ids)
df_records_ids.to_csv("human_uniprotIDs.tsv", sep="\t", index=False, header=False)
!head human_uniprotIDs.tsv

UP000005640_9606.fasta	   UP000005640_9606.idmapping
UP000005640_9606.gene2acc  UP000005640_9606.xml
A0A024R161
A0A024R1R8
A0A075B6F4
A0A075B6H5
A0A075B6H7
A0A075B6H8
A0A075B6H9
A0A075B6I0
A0A075B6I1
A0A075B6I3


## Orthologs retrievement

### From MetaPhors

In [7]:
!ls drive/MyDrive/TFG/QfO_input.tsv #Orthologs input for QfO

drive/MyDrive/TFG/QfO_input.tsv


#Phylogenetic Profiling (PP)
